In [3]:
import sys
from pathlib import Path
project_path = str(Path.cwd().parent)
sys.path.append(project_path)


import pymysql
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

import sys
import requests
from bs4 import BeautifulSoup
from tabs import tab1, tab2, tab3, tab4, tab5
from agents import get_user_agents

from schemas.general import GeneralCreateSchema
from schemas.subscriber import SubscriberCreateSchema
from schemas.financial import FinancialCreateSchema
from schemas.shareholder import ShareholderCreateSchema
from schemas.prediction import PredictionCreateSchema

from db import models
from sqlalchemy import and_ 
from settings import DB_URL, DELISTING
from utilities.session import session_scope
import re
from datetime import datetime

import requests
from bs4 import BeautifulSoup

from agents import get_user_agents
from utilities.time_measure import timeit


# @timeit
def scrape_company_codes(year=2021):

    current_year = datetime.now().year
    codes = []
    for year in range(year, current_year + 1):
        # '리스트가 없습니다.'라는 colspan=9 속성이 안나올때까지 순회하게 됨.
        page_data = None
        page = 1
        # while page_data is None:
        url = "http://www.ipostock.co.kr/sub03/ipo02.asp?str4=2022&str5=all"
        # url = f"http://www.ipostock.co.kr/sub03/ipo02.asp?str4={year}&str5=all&page={page}"
        # page += 1

    

        req = requests.get(url)
        soup = BeautifulSoup(req.content, "lxml", )

        page_data = soup.select_one('td[colspan="9"]')
        # if page_data is not None:
        #     break

        a_tags = soup.select('a[href*="view_pg"][class=""]')

        for tag in a_tags:
            href = tag["href"]
            code = re.search("code=(.+)&", href).group(1)
            codes.append(code)
    return codes



In [4]:
url = "http://www.ipostock.co.kr/sub03/ipo02.asp?str4=2022&str5=all"
# url = f"http://www.ipostock.co.kr/sub03/ipo02.asp?str4={year}&str5=all&page={page}"
# page += 1

from utilities import request_helper

req = requests.get(url)
soup = BeautifulSoup(req.content, "lxml", )

In [9]:
import bs4
from db.dbmanager import DBManager
# ㄷㅏㅇ ㅇㅣㄹ ㄱㅗㅇㅁㅗ ㅊㅓㄹㅎㅗㅣㄹㅡㄹ ㅈㅓㅇㅂㅗㄹㅡㄹ ㅇㅗㅎㅜㅇㅔ ㅇㅗㄹㄹㅣㄱㅔ ㄷㅗㅣㄴㅡㄴㄷㅔ ㅇㅗㅈㅓㄴㅇㅔ ㄱㅣㅇㅓㅂ ㅈㅓㅇㅂㅗㄹㅡㄹ ㅂㅕㄴㄱㅕㅇㅎㅏㄱㅗ ㅇㅗㅎㅜㅇㅔ ㄱㅗㅇㅁㅗ ㅊㅓㄹㅎㅗㅣㅅㅣ?
table = soup.select_one('#print > table >  tr:nth-child(4) > td > table >  tr:nth-child(3) > td > table >  tr:nth-child(4) > td > table')
codes = []
delisted_codes = []
for idx, tr in enumerate(table, 1):
    if isinstance(tr, bs4.element.Tag):
        td = tr.find('td', {'width': '120'})
        
        if td is not None:
            delisting = td.text.strip()
            td_name = tr.find('td', {'width': '*'})
            
            if delisting == '공모철회':
                if td_name is not None:
                    a_tag = td_name.select_one('table > tr > td:nth-child(2) > a[href*="view_pg"][class=""]')
                    href = a_tag.get('href')
                    code = re.search("code=(.+)&", href).group(1)
                    delisted_codes.append(code)
                    company_name = td_name.select_one('table > tr > td:nth-child(2) > a > font').text.strip().replace('.', '')
                    instance = DBManager()
                    if isinstance(company_name, str) and company_name > 2 :
                        read, ci_demand_forcast_date = instance.read(company_name)
                        
                        # DB에는 공모철회가 없지만 공모철회로 데이터를 바꾸어야함.
                        if read and ci_demand_forcast_date != '공모철회':
                            delisted = instance.delist(company_name)
                            # 성공적으로 종목을 공모 철회로 바꾼 경우. 
                            # 이미 되어 있는 경우 
                            continue
        
                    # 회사 이름으로 쿼리 때려서 2가지로 나뉨
                    # 1) 반환 값이 공모철회(0) -> 
                    # 2) 반환 값이 공모철회(X) -> ci_demand_forecast_? 를 공모철회로 변경
            else:
                a_tag = td_name.select_one('a[href*="view_pg"][class=""]')
                href = a_tag.get('href')
                code = re.search("code=(.+)&", href).group(1)
                codes.append(code)
return codes
print(codes)

유안타제12호스팩
자람테크놀로지
유안타제11호스팩
바이오인프라
미래에셋드림스팩1호
밀리의서재
['B202210113', 'B202206162', 'B202210112', 'B202209191', 'B202208181', 'B202209071', 'B202209073', 'B202206021', 'B202208021', 'B202207181', 'B202205271', 'B202205121', 'B202210131', 'B202205161']


In [40]:

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from db import models
from utilities.session import session_scope


ci_name = '래몽래인'
DELISTING = '공모철회'

engine = create_engine(DB_URL, pool_recycle=3600)
Session = sessionmaker(bind=engine)

with session_scope(Session) as session:
    result = session.query(models.CompanyInfoGeneral).all()
    for row in result:
        print(row)



In [117]:
DELISTING = 'a'
with session_scope(Session) as session:
    result = session.query(models.CompanyInfoGeneral)\
        .filter(and_(models.CompanyInfoGeneral.ci_name == ci_name, models.CompanyInfoGeneral.ci_demand_forecast_date != DELISTING))\
        .update({models.CompanyInfoGeneral.ci_demand_forecast_date: DELISTING})
    print(type(result), resul   t)
    
    session.commit()
    print(type(result), result)

<class 'int'> 2
<class 'int'> 2


In [37]:
ci_name = '래몽래인'
with session_scope(Session) as session:
    print(ci_name)
    result = session.query(models.CompanyInfoGeneral)\
        .filter(
            and_(
                models.CompanyInfoGeneral.ci_name == ci_name,
                # models.CompanyInfoGeneral.ci_demand_forecast_date != DELISTING,
            )
        )\
        .first()
    print(result)


래몽래인
None


In [4]:
for i in range(3):
    if 1:
        pass
    else:
        print(i)